In [1]:
import sys
from matplotlib import pyplot
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import SeparableConv2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.optimizers import SGD
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import os
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
import numpy as np

2021-11-20 03:25:07.606428: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2021-11-20 03:25:07.606471: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


Jupyter notebook to build, train, and deply CNN models for TF Lite

In [31]:
filename = "shallow_CNN" # Set the name for the model output

early_stop_training = True # Early stop training

#___________________________
training_batch_size = 20
training_epochs = 100
training_patience = 10

#___________________________
quantize_aware_training = True
quantize_training_epochs = 100
quantize_training_patience = 10

target_exponent = 5 # Target exponent bit size for custom floating-point 
target_mantissa = 1 # Target mantissa bit size for custom floating-point

max_degradation = 0.00

In [3]:
google_colab = False # Set True when using on google colab
if google_colab:
  from google.colab import drive
  drive.mount('/content/gdrive')

CNN ARCHITECTURE:

In [4]:
# define cnn model
def ConvPool_CNN_C():
    model = Sequential()

    model.add(Conv2D(96, (3, 3), activation='relu', padding = 'same', input_shape=(32, 32, 3)))
    model.add(Conv2D(96, (3, 3), activation='relu', padding = 'same'))
    model.add(Conv2D(96, (3, 3), activation='relu', padding = 'same'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides = 2))

    model.add(Conv2D(192, (3, 3), activation='relu', padding = 'same'))
    model.add(Conv2D(192, (3, 3), activation='relu', padding = 'same'))
    model.add(Conv2D(192, (3, 3), activation='relu', padding = 'same'))
    model.add(MaxPooling2D(pool_size=(3, 3), strides = 2))

    model.add(Conv2D(192, (3, 3), activation='relu', padding = 'same'))
    model.add(Conv2D(192, (1, 1), activation='relu'))
    model.add(Conv2D(10, (1, 1)))
    model.add(GlobalAveragePooling2D())
    model.add(Activation(activation='softmax'))

    # compile model
    #opt = SGD(learning_rate=0.001, momentum=0.9)
    opt = "adam"
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [5]:
# define cnn model
def shallow_CNN():
	model = Sequential()
	
	model.add(Conv2D(40, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.3))

	model.add(Conv2D(60, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.3))

	model.add(Conv2D(120, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.3))

	model.add(Flatten())
	model.add(Dense(10, activation='softmax'))
	# compile model
	#opt = SGD(learning_rate=0.001, momentum=0.9)
	opt = "adam"
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model

In [6]:
# define cnn model
def shallow_dropout_CNN():
	model = Sequential()
	
	model.add(Conv2D(40, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.3))

	model.add(Conv2D(60, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.3))

	model.add(Conv2D(120, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.3))

	model.add(Flatten())
	model.add(Dense(10, activation='softmax'))
	# compile model
	#opt = SGD(learning_rate=0.001, momentum=0.9)
	opt = "adam"
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model

In [7]:
# define cnn model
def shallow_SeparableCNN():
	model = Sequential()
	
	model.add(SeparableConv2D(40, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same', input_shape=(32, 32, 3)))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.3))

	model.add(SeparableConv2D(60, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.3))

	model.add(SeparableConv2D(120, (3, 3), activation='relu', kernel_initializer='he_uniform', padding='same'))
	model.add(BatchNormalization())
	model.add(MaxPooling2D((2, 2)))
	model.add(Dropout(0.3))

	model.add(Flatten())
	model.add(Dense(10, activation='softmax'))
	# compile model
	#opt = SGD(learning_rate=0.001, momentum=0.9)
	opt = "adam"
	model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
	return model

In [8]:
# define cnn model
def define_model_CNN():
	return shallow_CNN()

Quantized aware training method.

In [9]:
import struct

def bin2float(b):
    ''' Convert binary string to a float.

    Attributes:
        :b: Binary string to transform.
    '''
    h = int(b, 2).to_bytes(8, byteorder="big")
    return struct.unpack('>d', h)[0]


def float2bin(f):
    ''' Convert float to 64-bit binary string.

    Attributes:
        :f: Float number to transform.
    '''
    [d] = struct.unpack(">Q", struct.pack(">d", f))
    return f'{d:064b}'

In [10]:
def quantize_float (float_number, exponent_bits, mantissa_bits):
    exponent_sign = 1
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
    if mantissa_bits < 0:
      mantissa_bits = 0
    if exponent_bits < 0:
      exponent_bits = 0
      
    coefficient = float2bin(float_number)
    sign = int (coefficient[:1], 2)
    if 0 < mantissa_bits:
        custom_mantissa = int (coefficient[12 : 12 + mantissa_bits], 2)
    else:
        custom_mantissa = 0
    residual_mantissa = int (coefficient[12 + mantissa_bits:], 2)
    exponent = int (coefficient[1:12], 2) - 1023

    exponent_full_range = pow(2, exponent_bits - exponent_sign) - 1
    if exponent < - exponent_full_range:
        quantized_value = 0
    elif exponent > exponent_full_range:
        quantized_value = pow(-1, sign) * (1 + (1 - pow(2, - mantissa_bits))) * pow(2, exponent_full_range)
    else:
        if (pow (2, (52 - (mantissa_bits + 1))) - 1) < residual_mantissa:
            custom_mantissa += 1
            if (pow (2, mantissa_bits) - 1) < custom_mantissa:
                custom_mantissa = 0
                exponent += 1
    
        quantized_value = pow(-1, sign) * (1 + custom_mantissa * pow(2, - mantissa_bits)) * pow(2, exponent)
    return quantized_value

def quantize_model(model, exponent_bits, mantissa_bits):
  for layer in model.layers:
    if isinstance(layer, tf.keras.layers.Conv2D) or isinstance(layer, tf.keras.layers.SeparableConv2D):
      layer_weights = layer.get_weights()
      for index in range(len(layer_weights)):
        matrix = layer_weights[index]
        for weight_index, weight in np.ndenumerate(matrix):
          matrix[weight_index] = quantize_float(weight, exponent_bits, mantissa_bits)
        layer_weights[index] = matrix
      layer.set_weights(layer_weights)

Early stop callback.

In [11]:
monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=10, verbose=1, mode='auto', restore_best_weights=True)

In [32]:
mantissa_bits = target_mantissa
exponent_bits = target_exponent
class MyCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs = None):
    quantize_model(self.model, exponent_bits, mantissa_bits)
  def on_train_end(self, logs = None):
    quantize_model(self.model, exponent_bits, mantissa_bits)
  def on_batch_end(self, epoch, logs = None):
    quantize_model(self.model, exponent_bits, mantissa_bits)


In [13]:
if not os.path.exists(filename):
  os.mkdir(filename)

# load train and test dataset
def load_dataset():
	# load dataset
	(trainX, trainY), (testX, testY) = cifar10.load_data()
	# one hot encode target values
	trainY = to_categorical(trainY)
	testY = to_categorical(testY)
	return trainX, trainY, testX, testY

# scale pixels
def prep_pixels(train, test):
	# convert from integers to floats
	train_norm = train.astype('float32')
	test_norm = test.astype('float32')
	# normalize to range 0-1
	train_norm = train_norm / 255.0
	test_norm = test_norm / 255.0
	# return normalized images
	return train_norm, test_norm

# plot diagnostic learning curves
def summarize_diagnostics(history):
	# plot loss
	f = pyplot.figure()
	f.set_figwidth(10)
	f.set_figheight(20)
	
	pyplot.subplot(211)
	pyplot.title('Cross Entropy Loss')
	pyplot.plot(history.history['loss'], color='blue', label='train')
	pyplot.plot(history.history['val_loss'], color='orange', label='test')
	# plot accuracy
	pyplot.subplot(212)
	pyplot.title('Classification Accuracy')
	pyplot.plot(history.history['accuracy'], color='blue', label='train')
	pyplot.plot(history.history['val_accuracy'], color='orange', label='test')
	# save plot to file
	pyplot.savefig(filename + '/' + filename + '_plot.png')
	pyplot.close()

def print_model(model):
	for layer in model.layers:
		if isinstance(layer, tf.keras.layers.Conv2D) or isinstance(layer, tf.keras.layers.SeparableConv2D):
			layer_weights = layer.get_weights()
			print (layer_weights)

# run the test harness for evaluating a model
def run_test_harness():
	# load dataset
	trainX, trainY, testX, testY = load_dataset()
	# prepare pixel data
	trainX, testX = prep_pixels(trainX, testX)
	# define model
	model = define_model_CNN()
	# fit model
	history = model.fit(trainX, trainY, epochs=20, batch_size=64, validation_data=(testX, testY), verbose=1, callbacks=[MyCallback()])
	# evaluate model
	_, acc = model.evaluate(testX, testY, verbose=0)
	print('> %.3f' % (acc * 100.0))
	# learning curves
	summarize_diagnostics(history)
	# Save the entire model to a HDF5 file.
	model.save(filename + "/" + filename + '.h5')

In [14]:
# entry point, run the test harness
# load dataset
trainX, trainY, testX, testY = load_dataset()

# prepare pixel data
trainX, testX = prep_pixels(trainX, testX)

# define model
model = define_model_CNN()
# fit model

2021-11-20 03:25:13.963894: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2021-11-20 03:25:13.976475: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-20 03:25:13.977268: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce GTX 860M computeCapability: 5.0
coreClock: 1.0195GHz coreCount: 5 deviceMemorySize: 1.96GiB deviceMemoryBandwidth: 74.65GiB/s
2021-11-20 03:25:13.977395: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2021-11-20 03:25:13.977501: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libc

In [15]:
if not early_stop_training:
    history = model.fit(trainX, trainY, epochs=training_epochs, batch_size=training_batch_size, validation_data=(testX, testY), verbose=1)
else:
    monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=training_patience, verbose=1, mode='auto', restore_best_weights=True)
    history = model.fit(trainX, trainY, epochs=training_epochs, batch_size=training_batch_size, validation_data=(testX, testY), verbose=1, callbacks=[monitor])

# evaluate model
_, pre_acc = model.evaluate(testX, testY, verbose=0)
print('Pre-quantization training > %.3f' % (pre_acc * 100.0))

2021-11-20 03:25:14.564846: W tensorflow/core/framework/cpu_allocator_impl.cc:81] Allocation of 614400000 exceeds 10% of free system memory.


Epoch 1/100
2500/2500 [==============================] - 53s 21ms/step - loss: 1.8998 - accuracy: 0.2978 - val_loss: 1.6192 - val_accuracy: 0.4173
Epoch 2/100
2500/2500 [==============================] - 54s 21ms/step - loss: 1.5753 - accuracy: 0.4226 - val_loss: 1.5058 - val_accuracy: 0.4810
Epoch 3/100
2500/2500 [==============================] - 54s 21ms/step - loss: 1.4247 - accuracy: 0.4848 - val_loss: 1.2092 - val_accuracy: 0.5582
Epoch 4/100
2500/2500 [==============================] - 54s 22ms/step - loss: 1.3174 - accuracy: 0.5254 - val_loss: 1.0327 - val_accuracy: 0.6339
Epoch 5/100
2500/2500 [==============================] - 54s 21ms/step - loss: 1.2439 - accuracy: 0.5571 - val_loss: 1.0989 - val_accuracy: 0.6127
Epoch 6/100
2500/2500 [==============================] - 54s 21ms/step - loss: 1.1810 - accuracy: 0.5836 - val_loss: 1.0296 - val_accuracy: 0.6429
Epoch 7/100
2500/2500 [==============================] - 53s 21ms/step - loss: 1.1335 - accuracy: 0.6000 - val_loss: 1

In [16]:
# learning curves
summarize_diagnostics(history)

# Save the entire model to a HDF5 file.
model.save(filename + "/" + filename + '.h5')

In [33]:
print ("_______ Post-training quantization _______")
model = load_model(filename + "/" + filename + '.h5')
_, pre_acc = model.evaluate(testX, testY, verbose=0)
print('Original accuracy > %.3f' % (pre_acc * 100.0))

quantize_model(model, exponent_bits, mantissa_bits)
# evaluate model
_, current_acc = model.evaluate(testX, testY, verbose=0)
print_model (model)
print('Post-training quantization > %.3f' % (current_acc * 100.0))

print("exponent = " + str(exponent_bits) + ", mantissa = " + str(mantissa_bits))

if quantize_aware_training:
    print ("_______ Quantize aware training _______")
    quantize_loop = True
    while quantize_loop:
        print ("Starting...")
        model = load_model(filename + "/" + filename + '.h5')
        monitor = EarlyStopping(monitor='val_loss', min_delta=1e-3, patience=quantize_training_patience, verbose=1, mode='auto', restore_best_weights=True)
        history = model.fit(trainX, trainY, epochs=quantize_training_epochs, batch_size=training_batch_size, validation_data=(testX, testY), verbose=1, callbacks=[MyCallback(), monitor])
        # learning curves
        summarize_diagnostics(history)
        # evaluate model
        _, test_acc = model.evaluate(testX, testY, verbose=0)
        print('Quantize aware training > %.3f' % (current_acc * 100.0))
        
        if current_acc < test_acc:
            current_acc = test_acc
            model.save(filename + "/" + filename + '.h5')
            print_model (model)
            print('Save model > %.3f' % (current_acc * 100.0))
            quantize_loop = current_acc < pre_acc - max_degradation
        
    print('Ending quantized aware training > %.3f' % (current_acc * 100.0))

_______ Post-training quantization _______
Original accuracy > 76.660
[array([[[[-0.125 , -0.125 ,  0.375 , -0.5   , -0.5   , -0.5   ,
          -0.1875, -0.5   , -0.375 ,  0.25  , -0.125 ,  0.5   ,
           0.375 ,  0.    , -0.125 ,  0.375 , -0.125 ,  0.1875,
           0.    ,  0.    ],
         [ 0.1875,  0.5   ,  0.75  , -0.25  ,  0.125 ,  0.125 ,
           0.    , -0.5   ,  0.1875,  0.375 , -0.1875,  0.25  ,
           0.    ,  0.125 , -0.25  ,  0.375 ,  0.    , -0.125 ,
           0.375 ,  0.1875],
         [-0.5   ,  0.1875,  0.    , -0.125 ,  0.25  ,  0.    ,
           0.5   ,  0.5   , -0.375 ,  0.1875,  0.25  ,  0.5   ,
           0.75  , -0.375 ,  0.25  ,  0.25  ,  0.    ,  0.    ,
           0.1875, -0.375 ]],

        [[ 0.375 ,  0.25  ,  0.    , -0.75  , -0.375 , -0.1875,
           0.    , -0.375 , -0.5   , -0.5   ,  0.5   , -0.75  ,
          -0.5   ,  0.25  , -0.1875,  0.75  , -0.5   , -0.5   ,
           0.125 , -0.75  ],
         [ 0.5   ,  0.75  , -0.5   ,  0.   

2500/2500 [==============================] - 831s 332ms/step - loss: 0.9370 - accuracy: 0.6748 - val_loss: 0.7746 - val_accuracy: 0.7288
Epoch 2/100
2500/2500 [==============================] - 840s 336ms/step - loss: 0.9306 - accuracy: 0.6768 - val_loss: 0.7310 - val_accuracy: 0.7430
Epoch 3/100
2500/2500 [==============================] - 839s 336ms/step - loss: 0.9369 - accuracy: 0.6762 - val_loss: 0.8159 - val_accuracy: 0.7182
Epoch 4/100
2500/2500 [==============================] - 837s 335ms/step - loss: 0.9308 - accuracy: 0.6761 - val_loss: 0.7576 - val_accuracy: 0.7369
Epoch 5/100
2500/2500 [==============================] - 840s 336ms/step - loss: 0.9369 - accuracy: 0.6745 - val_loss: 0.7351 - val_accuracy: 0.7431
Epoch 6/100
2500/2500 [==============================] - 830s 332ms/step - loss: 0.9338 - accuracy: 0.6751 - val_loss: 0.7893 - val_accuracy: 0.7261
Epoch 7/100
2500/2500 [==============================] - 832s 333ms/step - loss: 0.9323 - accuracy: 0.6760 - val_loss:

2500/2500 [==============================] - 848s 339ms/step - loss: 0.9396 - accuracy: 0.6732 - val_loss: 0.7666 - val_accuracy: 0.7353
Epoch 2/100
2500/2500 [==============================] - 864s 346ms/step - loss: 0.9450 - accuracy: 0.6735 - val_loss: 0.7413 - val_accuracy: 0.7390
Epoch 3/100
2500/2500 [==============================] - 830s 332ms/step - loss: 0.9357 - accuracy: 0.6743 - val_loss: 0.7849 - val_accuracy: 0.7297
Epoch 4/100
2500/2500 [==============================] - 837s 335ms/step - loss: 0.9382 - accuracy: 0.6725 - val_loss: 0.8001 - val_accuracy: 0.7203
Epoch 5/100
2500/2500 [==============================] - 830s 332ms/step - loss: 0.9308 - accuracy: 0.6781 - val_loss: 0.7527 - val_accuracy: 0.7391
Epoch 6/100
2500/2500 [==============================] - 835s 334ms/step - loss: 0.9341 - accuracy: 0.6745 - val_loss: 0.7703 - val_accuracy: 0.7330
Epoch 7/100
2500/2500 [==============================] - 821s 328ms/step - loss: 0.9332 - accuracy: 0.6760 - val_loss:

2500/2500 [==============================] - 839s 336ms/step - loss: 0.9355 - accuracy: 0.6755 - val_loss: 0.7765 - val_accuracy: 0.7320
Epoch 2/100
2500/2500 [==============================] - 845s 338ms/step - loss: 0.9388 - accuracy: 0.6753 - val_loss: 0.7811 - val_accuracy: 0.7283
Epoch 3/100
2500/2500 [==============================] - 870s 348ms/step - loss: 0.9331 - accuracy: 0.6756 - val_loss: 0.7556 - val_accuracy: 0.7379
Epoch 4/100
2500/2500 [==============================] - 982s 393ms/step - loss: 0.9281 - accuracy: 0.6767 - val_loss: 0.7805 - val_accuracy: 0.7284
Epoch 5/100
2500/2500 [==============================] - 1094s 438ms/step - loss: 0.9308 - accuracy: 0.6787 - val_loss: 0.7786 - val_accuracy: 0.7282
Epoch 6/100
2500/2500 [==============================] - 896s 359ms/step - loss: 0.9358 - accuracy: 0.6753 - val_loss: 0.7636 - val_accuracy: 0.7344
Epoch 7/100
2500/2500 [==============================] - 868s 347ms/step - loss: 0.9311 - accuracy: 0.6791 - val_loss

2500/2500 [==============================] - 1032s 413ms/step - loss: 0.9326 - accuracy: 0.6763 - val_loss: 0.7818 - val_accuracy: 0.7279
Epoch 2/100
2500/2500 [==============================] - 898s 359ms/step - loss: 0.9346 - accuracy: 0.6761 - val_loss: 0.7757 - val_accuracy: 0.7297
Epoch 3/100
2500/2500 [==============================] - 980s 392ms/step - loss: 0.9274 - accuracy: 0.6787 - val_loss: 0.7588 - val_accuracy: 0.7360
Epoch 4/100
2500/2500 [==============================] - 895s 358ms/step - loss: 0.9384 - accuracy: 0.6738 - val_loss: 0.7465 - val_accuracy: 0.7383
Epoch 5/100
2500/2500 [==============================] - 908s 363ms/step - loss: 0.9331 - accuracy: 0.6743 - val_loss: 0.7866 - val_accuracy: 0.7242
Epoch 6/100
2500/2500 [==============================] - 873s 349ms/step - loss: 0.9370 - accuracy: 0.6742 - val_loss: 0.7282 - val_accuracy: 0.7449
Epoch 7/100
2500/2500 [==============================] - 894s 358ms/step - loss: 0.9327 - accuracy: 0.6757 - val_loss

KeyboardInterrupt: 

In [18]:
# Save the entire model to a HDF5 file.
model.save(filename + "/" + filename + '.h5')

_______ LOAD CNN MODEL FOR EVALUATION AND CONVERSION TO TF LITE FLOATING-POINT AND FIXED-POINT _______

In [19]:
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

img_path = "dog.jpg"
img = image.load_img(img_path, target_size=(32, 32))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = x.astype('float32')
x = x / 255.0

model = load_model(filename + "/" + filename + ".h5")

output_data = model.predict(x)

print('TensorFlow Predicted:', output_data)

print("Done!")

TensorFlow Predicted: [[4.21228498e-04 2.04306798e-06 1.91781923e-01 3.10171247e-02
  4.76600006e-02 6.01640165e-01 1.38121387e-02 1.13585785e-01
  8.98247981e-06 7.06499850e-05]]
Done!


In [20]:
def print_model(model):
  for layer in model.layers:
    if isinstance(layer, tf.keras.layers.Conv2D) or isinstance(layer, tf.keras.layers.SeparableConv2D):
      layer_weights = layer.get_weights()
      print (layer_weights)

In [21]:
quantize_model(model, exponent_bits, mantissa_bits)

print_model (model)
_, acc = model.evaluate(testX, testY, verbose=0)
print('> %.3f' % (acc * 100.0))

[array([[[[-1.2500000e-01, -1.2500000e-01,  3.7500000e-01,
          -5.0000000e-01, -5.0000000e-01, -5.0000000e-01,
          -1.8750000e-01, -5.0000000e-01, -3.7500000e-01,
           2.5000000e-01, -1.2500000e-01,  5.0000000e-01,
           3.7500000e-01, -1.5625000e-02, -1.2500000e-01,
           3.7500000e-01, -1.2500000e-01,  1.8750000e-01,
          -1.1718750e-02, -4.6875000e-02],
         [ 1.8750000e-01,  5.0000000e-01,  7.5000000e-01,
          -2.5000000e-01,  1.2500000e-01,  1.2500000e-01,
           4.6875000e-02, -5.0000000e-01,  1.8750000e-01,
           3.7500000e-01, -1.8750000e-01,  2.5000000e-01,
           9.3750000e-02,  1.2500000e-01, -2.5000000e-01,
           3.7500000e-01,  1.1718750e-02, -1.2500000e-01,
           3.7500000e-01,  1.8750000e-01],
         [-5.0000000e-01,  1.8750000e-01,  9.3750000e-02,
          -1.2500000e-01,  2.5000000e-01,  6.2500000e-02,
           5.0000000e-01,  5.0000000e-01, -3.7500000e-01,
           1.8750000e-01,  2.5000000e-01,  

In [22]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models, utils

if not os.path.exists(filename):
  print("Folder " + filename + " does not exist.")
  exit()

# Recreate the exact same model, including its weights and the optimizer
model = tf.keras.models.load_model(filename + "/" + filename + ".h5")

model.summary()

tf.keras.utils.plot_model(model, filename + "/" + filename + ".png", show_shapes=True, show_layer_names=True, expand_nested=True)

# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
#converter.optimizations = [tf.lite.Optimize.DEFAULT]
#converter.representative_dataset = representative_dataset
#converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.float32  # or tf.uint8
converter.inference_output_type = tf.float32  # or tf.uint8
tflite_model = converter.convert()

# Save the model.
with open(filename + "/" + filename + '.tflite', 'wb') as f:
  f.write(tflite_model)

print("Done!")

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 32, 32, 20)        560       
_________________________________________________________________
batch_normalization (BatchNo (None, 32, 32, 20)        80        
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 16, 16, 20)        0         
_________________________________________________________________
dropout (Dropout)            (None, 16, 16, 20)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 16, 16, 30)        5430      
_________________________________________________________________
batch_normalization_1 (Batch (None, 16, 16, 30)        120       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 8, 8, 30)          0

2021-11-20 08:49:01.997423: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /tmp/tmpqgj3p2bv/assets
Done!


2021-11-20 08:49:03.372524: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-20 08:49:03.373064: I tensorflow/core/grappler/devices.cc:69] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0
2021-11-20 08:49:03.373152: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2021-11-20 08:49:03.450046: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-11-20 08:49:03.450892: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55e3df266210 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2021-11-20 08:49:03.450907: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): GeForce GT

In [23]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.float32  # or tf.uint8
converter.inference_output_type = tf.float32  # or tf.uint8
tflite_model = converter.convert()

# Save the model.
with open(filename + "/" + filename + "_i8" + '.tflite', 'wb') as f:
  f.write(tflite_model)

print("Done!")

NameError: name 'representative_dataset' is not defined

In [ ]:

# Load TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path=filename + "/" + filename + "_i8" + '.tflite')
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [ ]:

interpreter.set_tensor(input_details[0]['index'], x)
interpreter.invoke()
output_data = interpreter.get_tensor(output_details[0]['index'])
print('TensorFlow Lite Predicted:', output_data)

TensorFlow Lite Predicted: [[0.         0.         0.05078125 0.08203125 0.17578125 0.20703125
  0.37890625 0.09765625 0.         0.00390625]]
